In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hope-english/english_hope_dev.csv
/kaggle/input/hope-english/english_hope_test.csv
/kaggle/input/hope-english/english_hope_train.csv


In [2]:
import pandas as pd

train = pd.read_csv("/kaggle/input/hope-english/english_hope_train.csv", sep="\t", header=None)
dev   = pd.read_csv("/kaggle/input/hope-english/english_hope_dev.csv", sep="\t", header=None)
test  = pd.read_csv("/kaggle/input/hope-english/english_hope_test.csv", sep="\t", header=None)

train.columns = ["raw"]
dev.columns = ["raw"]
test.columns = ["raw"]

In [3]:
def clean_split(df):
    df = df.copy()

    # split by ";" into max 3 parts
    parts = df["raw"].str.split(";", expand=True)
    
    # parts[0] = text  
    # parts[1] = label
    df["text"] = parts[0]
    df["label"] = parts[1]

    # drop rows without labels
    df = df.dropna(subset=["label"])
    
    # keep only text + label
    df = df[["text", "label"]]
    return df

train = clean_split(train)
dev = clean_split(dev)
test = clean_split(test)

print(train.head())
print(train.shape)



                                                text            label
0  these tiktoks radiate gay chaotic energy and i...  Non_hope_speech
1  @Champions Again He got killed for using false...  Non_hope_speech
2               It's not that all lives don't matter  Non_hope_speech
3  Is it really that difficult to understand? Bla...  Non_hope_speech
4  Whenever we say black isn't that racists?  Why...  Non_hope_speech
(22762, 2)


In [4]:
import re

def clean_text(t):
    t = t.lower()
    t = re.sub(r"http\S+|www\S+", "", t)       # remove URLs
    t = re.sub(r"@\w+", "", t)                # remove usernames
    t = re.sub(r"\s+", " ", t).strip()        # remove extra spaces
    return t

train["text"] = train["text"].apply(clean_text)
dev["text"] = dev["text"].apply(clean_text)
test["text"] = test["text"].apply(clean_text)


In [5]:
label_map = {
    "Hope_speech": 1,
    "Non_hope_speech": 0
}

train["label"] = train["label"].map(label_map)
dev["label"]   = dev["label"].map(label_map)
test["label"]  = test["label"].map(label_map)


In [6]:
train = train.dropna(subset=["label"])
dev   = dev.dropna(subset=["label"])
test  = test.dropna(subset=["label"])


In [7]:
print(train["label"].value_counts())

label
0.0    20700
1.0     1945
Name: count, dtype: int64


In [8]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"


2025-12-26 17:55:50.354588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766771750.823621      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766771750.936778      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766771752.038130      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766771752.038170      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766771752.038173      55 computation_placer.cc:177] computation placer alr

In [9]:
mpnet_model = SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2",
    device=device
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
t5_model = SentenceTransformer(
    "sentence-transformers/sentence-t5-base",
    device=device
)


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [11]:
roberta_name = "roberta-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_name)
roberta_model = AutoModel.from_pretrained(roberta_name).to(device)
roberta_model.eval()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [12]:
def encode_roberta(texts, batch_size=32):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        encoded = roberta_tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            output = roberta_model(**encoded)

        # CLS token
        cls_emb = output.last_hidden_state[:, 0, :]
        cls_emb = F.normalize(cls_emb, p=2, dim=1)

        all_embeddings.append(cls_emb.cpu())

    return torch.cat(all_embeddings).numpy()


In [13]:
train_texts = train["text"].tolist()
dev_texts   = dev["text"].tolist()
test_texts  = test["text"].tolist()


In [14]:
#MPNet
train_mpnet = mpnet_model.encode(train_texts, batch_size=32, show_progress_bar=True)
dev_mpnet   = mpnet_model.encode(dev_texts, batch_size=32)
test_mpnet  = mpnet_model.encode(test_texts, batch_size=32)

#T5 
train_t5 = t5_model.encode(train_texts, batch_size=32, show_progress_bar=True)
dev_t5   = t5_model.encode(dev_texts, batch_size=32)
test_t5  = t5_model.encode(test_texts, batch_size=32)

#Roberta CLS
train_rob = encode_roberta(train_texts)
dev_rob   = encode_roberta(dev_texts)
test_rob  = encode_roberta(test_texts)


Batches:   0%|          | 0/708 [00:00<?, ?it/s]

Batches:   0%|          | 0/708 [00:00<?, ?it/s]

In [15]:
#CNN(AGNOSTIC)
import torch
import torch.nn as nn
import torch.nn.functional as F

class HopeSpeechCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, input_dim)
        self.dropout = nn.Dropout(0.25)

        self.conv1 = nn.Conv1d(1, 64, kernel_size=5)
        self.pool1 = nn.MaxPool1d(4)

        self.conv2 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(4)

        self.conv3 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool3 = nn.MaxPool1d(4)

        # Dynamically compute output size
        self._dummy_forward(input_dim)

        self.fc_out = nn.Linear(self.flatten_dim, 2)

    def _dummy_forward(self, input_dim):
        with torch.no_grad():
            x = torch.zeros(1, input_dim)
            x = x.unsqueeze(1)
            x = self.pool1(F.relu(self.conv1(x)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.pool3(F.relu(self.conv3(x)))
            self.flatten_dim = x.numel()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = x.unsqueeze(1)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)
        return self.fc_out(x)


In [21]:
from sklearn.metrics import accuracy_score, f1_score
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate(model, loader,device):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            pred = torch.argmax(logits, dim=1)

            preds.extend(pred.cpu().numpy())
            labels.extend(y.cpu().numpy())

    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return acc, macro_f1


In [17]:
# def train_model(model, train_loader, dev_loader, epochs=10, lr=1e-4):
#     model.to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=lr)

#     best_f1 = 0.0

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         for X, y in train_loader:
#             X, y = X.to(device), y.to(device)

#             optimizer.zero_grad()
#             logits = model(X)
#             loss = criterion(logits, y)
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#         acc, f1 = evaluate(model, dev_loader)

#         print(f"\nEpoch {epoch+1}/{epochs}")
#         print(f"Train Loss: {total_loss/len(train_loader):.4f}")
#         print(f"Dev Acc   : {acc:.4f}")
#         print(f"Dev F1    : {f1:.4f}")

#         if f1 > best_f1:
#             best_f1 = f1
#             torch.save(model.state_dict(), "best_model.pt")
#             print(" Best model saved")

#     print(f"\nBest Dev Macro-F1: {best_f1:.4f}")


In [18]:
def train_model(
    model,
    train_loader,
    dev_loader,
    epochs=10,
    lr=1e-4,
    model_name="model",
    device="cuda"
):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_f1 = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        dev_acc, dev_f1 = evaluate(model, dev_loader, device)

        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_loss:.4f}")
        print(f"Dev Acc   : {dev_acc:.4f}")
        print(f"Dev F1    : {dev_f1:.4f}")

        if dev_f1 > best_f1:
            best_f1 = dev_f1
            torch.save(model.state_dict(), f"{model_name}.pt")
            print(" Best model saved")

    print(f"\n Best Dev Macro-F1 ({model_name}): {best_f1:.4f}")


In [19]:
from torch.utils.data import DataLoader, TensorDataset

def make_loaders(train_emb, dev_emb, test_emb, y_train, y_dev, y_test, batch_size=32):

    train_ds = TensorDataset(
        torch.tensor(train_emb, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long)
    )

    dev_ds = TensorDataset(
        torch.tensor(dev_emb, dtype=torch.float32),
        torch.tensor(y_dev, dtype=torch.long)
    )

    test_ds = TensorDataset(
        torch.tensor(test_emb, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.long)
    )

    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(dev_ds, batch_size=batch_size),
        DataLoader(test_ds, batch_size=batch_size)
    )


In [22]:
train_loader_mpnet, dev_loader_mpnet, test_loader_mpnet = make_loaders(
    train_mpnet, dev_mpnet, test_mpnet,
    train["label"], dev["label"], test["label"]
)

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)

train_model(
    model_mpnet,
    train_loader_mpnet,
    dev_loader_mpnet,
    epochs=10,
    lr=1e-4,
    model_name="cnn_mpnet",
    device=device
)




Epoch 1/10
Train Loss: 0.2687
Dev Acc   : 0.9039
Dev F1    : 0.4748
 Best model saved

Epoch 2/10
Train Loss: 0.1829
Dev Acc   : 0.9247
Dev F1    : 0.7465
 Best model saved

Epoch 3/10
Train Loss: 0.1687
Dev Acc   : 0.9216
Dev F1    : 0.7705
 Best model saved

Epoch 4/10
Train Loss: 0.1580
Dev Acc   : 0.9208
Dev F1    : 0.7707
 Best model saved

Epoch 5/10
Train Loss: 0.1482
Dev Acc   : 0.9127
Dev F1    : 0.7734
 Best model saved

Epoch 6/10
Train Loss: 0.1356
Dev Acc   : 0.9254
Dev F1    : 0.7785
 Best model saved

Epoch 7/10
Train Loss: 0.1228
Dev Acc   : 0.9251
Dev F1    : 0.7801
 Best model saved

Epoch 8/10
Train Loss: 0.1104
Dev Acc   : 0.9131
Dev F1    : 0.7707

Epoch 9/10
Train Loss: 0.0959
Dev Acc   : 0.9205
Dev F1    : 0.7802
 Best model saved

Epoch 10/10
Train Loss: 0.0821
Dev Acc   : 0.7887
Dev F1    : 0.6612

 Best Dev Macro-F1 (cnn_mpnet): 0.7802


In [ ]:
# #MPNET TRAINING
# train_loader, dev_loader, test_loader = make_loaders(
#     train_mpnet, dev_mpnet, test_mpnet,
#     train["label"], dev["label"], test["label"]
# )

# model = HopeSpeechCNN(input_dim=768)
# train_model(model, train_loader, dev_loader)


In [26]:
# T5 TRAINING

model_t5 = HopeSpeechCNN(input_dim=768).to(device)

train_loader_t5, dev_loader_t5, test_loader_t5 = make_loaders(
    train_t5, dev_t5, test_t5,
    train["label"], dev["label"], test["label"]
)

train_model(
    model_t5,
    train_loader_t5,
    dev_loader_t5,
    epochs=10,
    lr=1e-4,
    model_name="cnn_t5",
    device=device
)





Epoch 1/10
Train Loss: 0.2889
Dev Acc   : 0.9039
Dev F1    : 0.4748
 Best model saved

Epoch 2/10
Train Loss: 0.1887
Dev Acc   : 0.8993
Dev F1    : 0.7639
 Best model saved

Epoch 3/10
Train Loss: 0.1797
Dev Acc   : 0.8823
Dev F1    : 0.7483

Epoch 4/10
Train Loss: 0.1769
Dev Acc   : 0.8940
Dev F1    : 0.7624

Epoch 5/10
Train Loss: 0.1747
Dev Acc   : 0.8862
Dev F1    : 0.7569

Epoch 6/10
Train Loss: 0.1732
Dev Acc   : 0.8816
Dev F1    : 0.7528

Epoch 7/10
Train Loss: 0.1713
Dev Acc   : 0.8477
Dev F1    : 0.7176

Epoch 8/10
Train Loss: 0.1690
Dev Acc   : 0.9078
Dev F1    : 0.7788
 Best model saved

Epoch 9/10
Train Loss: 0.1683
Dev Acc   : 0.7936
Dev F1    : 0.6654

Epoch 10/10
Train Loss: 0.1669
Dev Acc   : 0.7898
Dev F1    : 0.6631

 Best Dev Macro-F1 (cnn_t5): 0.7788


In [27]:
# model_rob = HopeSpeechCNN(input_dim=768).to(device)

# train_loader, dev_loader, test_loader = make_loaders(
#     train_rob, dev_rob, test_rob,
#     train["label"], dev["label"], test["label"]
# )

# train_model(model_rob, train_loader, dev_loader)

# ROBERTA TRAINING

model_rob = HopeSpeechCNN(input_dim=768).to(device)

train_loader_rob, dev_loader_rob, test_loader_rob = make_loaders(
    train_rob, dev_rob, test_rob,
    train["label"], dev["label"], test["label"]
)

train_model(
    model_rob,
    train_loader_rob,
    dev_loader_rob,
    epochs=10,
    lr=1e-4,
    model_name="cnn_roberta",
    device=device
)





Epoch 1/10
Train Loss: 0.3183
Dev Acc   : 0.9039
Dev F1    : 0.4748
 Best model saved

Epoch 2/10
Train Loss: 0.2872
Dev Acc   : 0.9039
Dev F1    : 0.4748

Epoch 3/10
Train Loss: 0.2704
Dev Acc   : 0.9039
Dev F1    : 0.4748

Epoch 4/10
Train Loss: 0.2375
Dev Acc   : 0.9042
Dev F1    : 0.6825
 Best model saved

Epoch 5/10
Train Loss: 0.2144
Dev Acc   : 0.8883
Dev F1    : 0.7114
 Best model saved

Epoch 6/10
Train Loss: 0.2088
Dev Acc   : 0.8767
Dev F1    : 0.7226
 Best model saved

Epoch 7/10
Train Loss: 0.2058
Dev Acc   : 0.8488
Dev F1    : 0.6997

Epoch 8/10
Train Loss: 0.2028
Dev Acc   : 0.8883
Dev F1    : 0.7325
 Best model saved

Epoch 9/10
Train Loss: 0.2019
Dev Acc   : 0.8933
Dev F1    : 0.7376
 Best model saved

Epoch 10/10
Train Loss: 0.2006
Dev Acc   : 0.9028
Dev F1    : 0.7455
 Best model saved

 Best Dev Macro-F1 (cnn_roberta): 0.7455


In [28]:
import numpy as np

# Fusion: MPNet + T5 + RoBERTa
X_train_fused = np.concatenate([train_mpnet, train_t5, train_rob], axis=1)
X_dev_fused   = np.concatenate([dev_mpnet, dev_t5, dev_rob], axis=1)
X_test_fused  = np.concatenate([test_mpnet, test_t5, test_rob], axis=1)

print(X_train_fused.shape)  # (N, 2304)


(22645, 2304)


In [29]:
# FUSION TRAINING

input_dim = X_train_fused.shape[1]  # 2304

model_fusion = HopeSpeechCNN(input_dim=input_dim).to(device)

train_loader_fus, dev_loader_fus, test_loader_fus = make_loaders(
    X_train_fused,
    X_dev_fused,
    X_test_fused,
    train["label"],
    dev["label"],
    test["label"]
)

train_model(
    model_fusion,
    train_loader_fus,
    dev_loader_fus,
    epochs=10,
    lr=1e-4,
    model_name="cnn_fusion",
    device=device
)



Epoch 1/10
Train Loss: 0.2387
Dev Acc   : 0.9265
Dev F1    : 0.7668
 Best model saved

Epoch 2/10
Train Loss: 0.1730
Dev Acc   : 0.9283
Dev F1    : 0.7622

Epoch 3/10
Train Loss: 0.1660
Dev Acc   : 0.9297
Dev F1    : 0.7748
 Best model saved

Epoch 4/10
Train Loss: 0.1601
Dev Acc   : 0.9318
Dev F1    : 0.7765
 Best model saved

Epoch 5/10
Train Loss: 0.1564
Dev Acc   : 0.9251
Dev F1    : 0.7958
 Best model saved

Epoch 6/10
Train Loss: 0.1496
Dev Acc   : 0.9304
Dev F1    : 0.7980
 Best model saved

Epoch 7/10
Train Loss: 0.1449
Dev Acc   : 0.9318
Dev F1    : 0.7939

Epoch 8/10
Train Loss: 0.1380
Dev Acc   : 0.8908
Dev F1    : 0.7567

Epoch 9/10
Train Loss: 0.1322
Dev Acc   : 0.8958
Dev F1    : 0.7644

Epoch 10/10
Train Loss: 0.1252
Dev Acc   : 0.9032
Dev F1    : 0.7675

 Best Dev Macro-F1 (cnn_fusion): 0.7980


In [ ]:
# train_loader, dev_loader, test_loader = make_loaders(
#     X_train_fused,
#     X_dev_fused,
#     X_test_fused,
#     train["label"],
#     dev["label"],
#     test["label"]
# )


In [ ]:
# input_dim = X_train_fused.shape[1]

# model_fusion = HopeSpeechCNN(input_dim=input_dim).to(device)

# train_model(
#     model_fusion,
#     train_loader,
#     dev_loader,
#     epochs=10,
#     lr=1e-4
# )


In [30]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
import torch
import numpy as np

def evaluate_on_test(model, test_loader, device):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")
    prec = precision_score(all_labels, all_preds, average="macro")
    rec  = recall_score(all_labels, all_preds, average="macro")

    print("Accuracy      :", round(acc, 4))
    print("Macro-F1      :", round(f1, 4))
    print("Macro-Precision:", round(prec, 4))
    print("Macro-Recall  :", round(rec, 4))
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, digits=4))

    return acc, f1, prec, rec


In [32]:
# MPNet TEST EVALUATION

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet.pt"))
model_mpnet.to(device)

evaluate_on_test(model_mpnet, test_loader_mpnet, device)



Accuracy      : 0.9233
Macro-F1      : 0.7818
Macro-Precision: 0.7588
Macro-Recall  : 0.8115

Classification Report:

              precision    recall  f1-score   support

           0     0.9683    0.9469    0.9575      2581
           1     0.5493    0.6761    0.6062       247

    accuracy                         0.9233      2828
   macro avg     0.7588    0.8115    0.7818      2828
weighted avg     0.9317    0.9233    0.9268      2828



(0.9232673267326733,
 0.7818316265857999,
 0.7588231920927517,
 0.8115165794257946)

In [34]:
# T5 TEST EVALUATION

model_t5 = HopeSpeechCNN(input_dim=768).to(device)
model_t5.load_state_dict(torch.load("cnn_t5.pt"))
model_t5.to(device)

evaluate_on_test(model_t5, test_loader_t5, device)


Accuracy      : 0.9028
Macro-F1      : 0.7619
Macro-Precision: 0.7208
Macro-Recall  : 0.8406

Classification Report:

              precision    recall  f1-score   support

           0     0.9761    0.9159    0.9450      2581
           1     0.4655    0.7652    0.5789       247

    accuracy                         0.9028      2828
   macro avg     0.7208    0.8406    0.7619      2828
weighted avg     0.9315    0.9028    0.9131      2828



(0.9027581329561527, 0.761949874485722, 0.7207850451322646, 0.8405531233382535)

In [36]:
# ROBERTA TEST EVALUATION

model_rob = HopeSpeechCNN(input_dim=768).to(device)
model_rob.load_state_dict(torch.load("cnn_roberta.pt"))
model_rob.to(device)

evaluate_on_test(model_rob, test_loader_rob, device)


Accuracy      : 0.9098
Macro-F1      : 0.7436
Macro-Precision: 0.7238
Macro-Recall  : 0.7694

Classification Report:

              precision    recall  f1-score   support

           0     0.9608    0.9396    0.9500      2581
           1     0.4868    0.5992    0.5372       247

    accuracy                         0.9098      2828
   macro avg     0.7238    0.7694    0.7436      2828
weighted avg     0.9194    0.9098    0.9140      2828



(0.9098302687411598, 0.7436270266330828, 0.7238093252147803, 0.769374297066542)

In [38]:
# FUSION TEST EVALUATION

input_dim = X_test_fused.shape[1]

model_fusion = HopeSpeechCNN(input_dim=input_dim).to(device)
model_fusion.load_state_dict(torch.load("cnn_fusion.pt"))
model_fusion.to(device)

evaluate_on_test(model_fusion, test_loader_fus, device)


Accuracy      : 0.9318
Macro-F1      : 0.7856
Macro-Precision: 0.7861
Macro-Recall  : 0.785

Classification Report:

              precision    recall  f1-score   support

           0     0.9624    0.9628    0.9626      2581
           1     0.6098    0.6073    0.6085       247

    accuracy                         0.9318      2828
   macro avg     0.7861    0.7850    0.7856      2828
weighted avg     0.9316    0.9318    0.9317      2828



(0.9317538896746818,
 0.7855689511774174,
 0.7860941603219285,
 0.7850462818447483)

In [39]:
results = {}


In [40]:
def evaluate_and_store(name, model, dev_loader, test_loader, device):
    from sklearn.metrics import f1_score
    import torch

    def eval_loader(loader):
        model.eval()
        preds, labels = [], []

        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                preds.extend(torch.argmax(logits, 1).cpu().numpy())
                labels.extend(y.cpu().numpy())

        return f1_score(labels, preds, average="macro")

    dev_f1 = eval_loader(dev_loader)
    test_f1 = eval_loader(test_loader)

    results[name] = {
        "Dev Macro-F1": round(dev_f1, 4),
        "Test Macro-F1": round(test_f1, 4)
    }

    print(f"{name}")
    print(" Dev Macro-F1 :", round(dev_f1, 4))
    print(" Test Macro-F1:", round(test_f1, 4))
    print("-" * 40)


In [42]:
model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet.pt"))

evaluate_and_store(
    "MPNet-CNN",
    model_mpnet,
    dev_loader_mpnet,
    test_loader_mpnet,
    device
)


MPNet-CNN
 Dev Macro-F1 : 0.7802
 Test Macro-F1: 0.7818
----------------------------------------


In [44]:
#t5
model_t5 = HopeSpeechCNN(input_dim=768).to(device)
model_t5.load_state_dict(torch.load("cnn_t5.pt"))

evaluate_and_store(
    "Sentence-T5-CNN",
    model_t5,
    dev_loader_t5,
    test_loader_t5,
    device
)

#roberta
model_rob = HopeSpeechCNN(input_dim=768).to(device)
model_rob.load_state_dict(torch.load("cnn_roberta.pt"))

evaluate_and_store(
    "RoBERTa-CNN",
    model_rob,
    dev_loader_rob,
    test_loader_rob,
    device
)

#fusion
input_dim = X_test_fused.shape[1]

model_fusion = HopeSpeechCNN(input_dim=input_dim).to(device)
model_fusion.load_state_dict(torch.load("cnn_fusion.pt"))

evaluate_and_store(
    "Fusion-CNN (MPNet+T5+RoBERTa)",
    model_fusion,
    dev_loader_fus,
    test_loader_fus,
    device
)




Sentence-T5-CNN
 Dev Macro-F1 : 0.7788
 Test Macro-F1: 0.7619
----------------------------------------
RoBERTa-CNN
 Dev Macro-F1 : 0.7455
 Test Macro-F1: 0.7436
----------------------------------------
Fusion-CNN (MPNet+T5+RoBERTa)
 Dev Macro-F1 : 0.798
 Test Macro-F1: 0.7856
----------------------------------------


In [45]:
import pandas as pd

df_results = pd.DataFrame(results).T
df_results


,Dev Macro-F1,Test Macro-F1
MPNet-CNN,0.7802,0.7818
Sentence-T5-CNN,0.7788,0.7619
RoBERTa-CNN,0.7455,0.7436
Fusion-CNN (MPNet+T5+RoBERTa),0.7980,0.7856
